In [2]:
# importing libraries

import nltk
import pandas as pd
import re
import string
import numpy as np

# import model libraries
import tensorflow as tf

# for visualizatiobn
import matplotlib.pyplot as plt

# NLTK libraries
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import twitter_samples
from sklearn.model_selection import train_test_split

#This function can only be called before any Graphs, Ops, or Tensors have been created.
#It can be used at the beginning of the program for complex migration projects from TensorFlow 1.x to 2.x.
from tensorflow.python.framework.ops import disable_eager_execution
disable_eager_execution()

In [3]:
# downloading daatsets

nltk.download('twitter_samples')
nltk.download('stopwords')

[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\rajka\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rajka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# getting positive and negative dataset

all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

print("positive tweets\n")
for _ in range(5):
  print(all_positive_tweets[_])

print()
print("negative tweets\n")
for _ in range(5):
  print(all_negative_tweets[_])

positive tweets

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!
@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!
@97sides CONGRATS :)
yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days

negative tweets

hopeless for tmr :(
Everything in the kids section of IKEA is so cute. Shame I'm nearly 19 in 2 months :(
@Hegelbon That heart sliding into the waste basket. :(
“@ketchBurning: I hate Japanese call him "bani" :( :(”

Me too
Dang starting next week I have "work" :(


In [5]:
# mixed data

main_data = all_positive_tweets + all_negative_tweets

# make labels for positive tweets its '1' and for negative tweets its '0'
main_labels = np.append(np.ones((len(all_positive_tweets), 1)), np.zeros((len(all_negative_tweets), 1)), axis = 0)

print("main_data\n")
for _ in range(5):
  print(main_data[_])

print("\nprint labels")
print(main_labels)

print(len(main_data))

main_data

#FollowFriday @France_Inte @PKuchly57 @Milipol_Paris for being top engaged members in my community this week :)
@Lamb2ja Hey James! How odd :/ Please call our Contact Centre on 02392441234 and we will be able to assist you :) Many thanks!
@DespiteOfficial we had a listen last night :) As You Bleed is an amazing track. When are you in Scotland?!
@97sides CONGRATS :)
yeaaaah yippppy!!!  my accnt verified rqst has succeed got a blue tick mark on my fb profile :) in 15 days

print labels
[[1.]
 [1.]
 [1.]
 ...
 [0.]
 [0.]
 [0.]]
10000


In [6]:
train_x, test_x, train_y, test_y = train_test_split(main_data, main_labels, test_size = 0.30, random_state = 55)

In [7]:
# whole methods and fcuntions for TF model training!

class MyTweetLogisticRegressionModel:
  def __transform_data(self, tweets):
    data = []
    for tweet in tweets:
      data.append(self.extract_features(tweet))
      #print(data)
    return (np.asarray(data))


  def fit(self, x, y, alpha = 1e-4, epochs = 20, threshold_val = 0.5, record_epoch = False):
    self.threshold_val = threshold_val
    self._freq = self.get_freqs(x, y)
    self._shape = 2
    self._inputs = self.__transform_data(x)
    self._tlen = len(y)

    # return sample of standard normal distribution
    self._bias = tf.Variable(np.random.randn(1), name = "Bias")
    self._weights = tf.Variable(np.random.randn(1, self._shape), name = "Weight")
    self._tinit = tf.compat.v1.global_variables_initializer()
    
    self._targets = np.asarray(y)
    self.alpha = alpha
    self.epochs = epochs
    self._repoch = record_epoch
    self.__trainModel()
    return


  def __initSaver(self):
    saver = tf.compat.v1.train.Saver()
    self._saver = saver
    self._sesspath = 'TSession'
    return


  def threshold(self, val):
    if (val >= self.threshold_val):
      return (1)
    return (0)


  def process_tweet(self, tweet):
    stemmer = PorterStemmer()
    stopwords_english = stopwords.words('english')
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case = False, strip_handles = True, reduce_len = True)
    tweet_tokens = tokenizer.tokenize(tweet)
    tweets_clean = []
    # cleaning the tweets
    for word in tweet_tokens:
      if word not in stopwords_english and word not in string.punctuation:
        tweets_clean.append(stemmer.stem(word))
        #print(tweets_clean)
    return (tweets_clean)


  def predict(self, indata):
    with tf.compat.v1.Session() as sess:
      self._saver.restore(sess,save_path = self._sesspath)
      indata = self.__transform_data(indata)
      return sess.run(tf.nn.sigmoid(tf.add(tf.matmul(a = indata, b = self._weights, transpose_b = True), self._bias)))
    print("Failed To Retrieve Session")
    return


  def get_accuracy(self, x, y):
    if len(x) != len(y):
      print("Dimension are different.")
      return
    return (((x == y).sum()) / len(y))


  def get_freqs(self, tweets, ys):
    freqs = dict()
    yslist = np.squeeze(ys).tolist()
    for y, tweet in zip(yslist, tweets):
      for word in self.process_tweet(tweet):
        pair = (word, y)
        if pair not in freqs:
          freqs[pair] = 0
        freqs[pair] += 1
    return freqs


  def extract_features(self, tweet):
    freqs = self._freq
    word_l = self.process_tweet(tweet)
    x = np.zeros((1, 2))
    for word in word_l: 
      if (word, 1) in freqs:
        x[0, 0] += freqs[word, 1]
      if (word, 0) in freqs:
        x[0, 1] += freqs[word, 0]
    assert(x.shape == (1, 2))
    return x[0]


  def getAccDets(self):
    if self._repoch:
      return self.__accurary_det
    return


  def __trainModel(self):
    if self._repoch:
      self._err = []
      self._precs = []
      self.__accurary_det = []
    
    # model of logistic regreesion of tensorflow.
    logi = tf.nn.sigmoid(tf.add(tf.matmul(a = self._inputs, b = self._weights, transpose_b = True), self._bias))
    err = tf.nn.sigmoid_cross_entropy_with_logits(logits = logi, labels = self._targets)
    temp = tf.compat.v1.train.GradientDescentOptimizer(learning_rate = self.alpha, name='GradientDescent').minimize(err)
    self.__initSaver()
    with tf.compat.v1.Session() as sess:
      sess.run(self._tinit)
      print("Bias", sess.run(self._bias))
      print("Weights", sess.run(self._weights))
      for epoch in range(self.epochs):
        sess.run(temp)
        __preds = sess.run(logi)
        acc = ((__preds == self._targets).sum()) / self._tlen
        if self._repoch:
          self.__accurary_det.append(acc)
        if epoch % 100 == 0:
          print("Acc:", acc)
        # print(self.__accurary_det)
      self._saver.save(sess, self._sesspath)
    return

In [8]:
tf_lr_model = MyTweetLogisticRegressionModel()

tf_lr_model.fit(x = train_x, y = train_y, alpha = 0.01, record_epoch = True, epochs = 500)

Bias [2.20542729]
Weights [[-0.92188684 -0.74097283]]
Acc: 0.45385714285714285
Acc: 0.4572857142857143
Acc: 0.49814285714285716
Acc: 0.49814285714285716
Acc: 0.49814285714285716
